In [32]:
import pandas as pd

In [33]:
med = pd.read_csv('medicine_new.csv')
gen = pd.read_csv('generic_new.csv')

In [34]:
"""
Medicine data transformation:

brand_name, generic_name_is, generic
brand_name, type_of, type
brand_name, has_dosage_form, dosage_form
brand_name, has_strength_of, strength
brand_name, manufactured_by, manufacturer
brand_name1, alternative_brand, brand_name2


Generic data transformation:

generic_name, pharmacology description, pharmacology_description
generic_name, side effects, side_effects_description

"""

'\nMedicine data transformation:\n\nbrand_name, generic_name_is, generic\nbrand_name, type_of, type\nbrand_name, has_dosage_form, dosage_form\nbrand_name, has_strength_of, strength\nbrand_name, manufactured_by, manufacturer\n\n\nGeneric data transformation:\n\ngeneric_name, pharmacology description, pharmacology_description\ngeneric_name, side effects, side_effects_description\n\n'

In [48]:
# Create a list to store the triples
triples = []

# Iterate through each row in the DataFrame
for index, row in med.iterrows():
    brand_name = row['brand name']
    generic = row['generic']
    med_type = row['type']
    dosage_form = row['dosage form']
    strength = row['strength']
    manufacturer = row['manufacturer']
    
    # Append the triples to the list
    triples.append([brand_name, 'generic name is', generic])
    triples.append([brand_name, 'type of', med_type])
    triples.append([brand_name, 'has dosage form', dosage_form])
    triples.append([brand_name, 'has strength of', strength])
    triples.append([brand_name, 'manufactured by', manufacturer])

# Iterate through each row in the 'gen' DataFrame
for index, row in gen.iterrows():
    generic_name = row['generic name']
    pharmacology_description = row['pharmacology description']
    side_effects_description = row['side effects description']
    
    # Append the triples to the list
    triples.append([generic_name, 'pharmacology description', pharmacology_description])
    triples.append([generic_name, 'side effects', side_effects_description])
    
# Get unique generics from the 'generic' column
unique_generics = med['generic'].unique()

# Iterate through each unique generic
for target_generic in unique_generics:
    # Use boolean indexing to filter the brand names for the specified generic
    filtered_data = med[med['generic'] == target_generic]

    # Reset the index of the filtered DataFrame
    filtered_data.reset_index(drop=True, inplace=True)

    # Create a list to store the triples for the current generic
    all_triples = []

    # Iterate through brand names to create the triples
    for i in range(len(filtered_data)):
        for j in range(i + 1, len(filtered_data)):
            brand1 = filtered_data.at[i, 'brand name']
            brand2 = filtered_data.at[j, 'brand name']

            # Append the triple (brand_name, alternative brand, brand_name)
            all_triples.append([brand1, 'alternative brand', brand2])

    # Extend the list of all triples with the triples for the current generic
    triples.extend(all_triples)

# Create a new DataFrame from the list of triples
df = pd.DataFrame(triples, columns=['Subject', 'Predicate', 'Object'])

# Remove any duplicate
df = df.drop_duplicates()

# Save the new DataFrame to a CSV file
df.to_csv('dictionary.csv', index=False, header=False)

print("Triples saved to dictionary.csv")

Triples saved to dictionary.csv


In [49]:
# column names
column_names = ['Subject', 'Predicate', 'Object']
df = pd.read_csv('dictionary.csv', header=None, names=column_names)

In [50]:
df.head()

,Subject,Predicate,Object
0,A-Cold,generic name is,Bromhexine Hydrochloride
1,A-Cold,type of,allopathic
2,A-Cold,has dosage form,Syrup
3,A-Cold,has strength of,4 mg/5 ml
4,A-Cold,manufactured by,ACME Laboratories Ltd.


In [53]:
import pandas as pd

# column names
column_names = ['Subject', 'Predicate', 'Object']
df = pd.read_csv('dictionary.csv', header=None, names=column_names)

def search_brand_info(brand_name):
    # Initialize variables to store the information
    generic_name = None
    brand_type = None
    dosage_form = None
    strength = None
    manufacturer = None
    pharmacology_description = None
    side_effects_description = None

    # Filter rows where the Subject matches the brand_name
    brand_rows = df[df['Subject'] == brand_name]

    # Loop through the filtered rows and extract information
    for _, row in brand_rows.iterrows():
        predicate = row['Predicate']
        obj = row['Object']
        if predicate == 'generic name is':
            generic_name = obj
        elif predicate == 'type of':
            brand_type = obj
        elif predicate == 'has dosage form':
            dosage_form = obj
        elif predicate == 'has strength of':
            strength = obj
        elif predicate == 'manufactured by':
            manufacturer = obj

    # Find the pharmacology description and side effects description based on generic name
    generic_rows = df[df['Subject'] == generic_name]
    for _, row in generic_rows.iterrows():
        predicate = row['Predicate']
        obj = row['Object']
        if predicate == 'pharmacology description':
            pharmacology_description = obj
        elif predicate == 'side effects':
            side_effects_description = obj

    # Create a dictionary to store the retrieved information
    brand_info = {
        'brand name': brand_name,
        'generic name': generic_name,
        'type of': brand_type,
        'has dosage form': dosage_form,
        'has strength of': strength,
        'manufactured by': manufacturer,
        'pharmacology description': pharmacology_description,
        'side effects': side_effects_description
    }

    return brand_info

# Example usage:
searched_brand = 'A-Cold'
result = search_brand_info(searched_brand)

# Print the retrieved information
for key, value in result.items():
    print(f"{key}: {value}")


brand name: A-Cold
generic name: Bromhexine Hydrochloride
type of: allopathic
has dosage form: Syrup
has strength of: 4 mg/5 ml
manufactured by: ACME Laboratories Ltd.
pharmacology description: Bromhexine Hydrochloride is an oral mucolytic agent. It disrupts the structure of acid mucopolysaccharide fibres in mucoid sputum and produces less viscous mucus, which makes easier the expectoration. It is rapidly & completely absorbed from the gastrointestinal tract. It is widely distributed into the body tissues.
side effects: Gastrointestinal side effects may occur occasionally. A transient rise in serum aminotransferase values has been reported. Other reported side effects include-headache, vertigo (dizziness) and allergic reactions.
